# Problem 1

## For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
   ### 1. Start by creating a new Notebook for this assignment.
   ### 2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
print('Downloaded Library')

Downloaded Library


In [36]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(URL).text

In [4]:
soup = BeautifulSoup(source, 'xml')
table = soup.find('table')

In [10]:
#Transform Data into DataFrame
column_names = ['PostalCode', 'Borough', 'Neighborhood']
CA_Post = pd.DataFrame(columns=column_names)
CA_Post

,PostalCode,Borough,Neighborhood


In [12]:
for tr_cell in table.find_all('tr'):
    row_data = []
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        CA_Post.loc[len(CA_Post)] = row_data
CA_Post.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [18]:
CA_Post = CA_Post[CA_Post['Borough'] !='Not assigned']
CA_Post.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [21]:
temp_df=CA_Post.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
temp_df.head()

,PostalCode,Neighborhood_joined
0,M1B,"Malvern, Rouge"
1,M1C,"Rouge Hill, Port Union, Highland Creek"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [23]:
CA_pst = pd.merge(CA_Post, temp_df, on = 'PostalCode')
CA_pst.drop (['Neighborhood'], axis=1, inplace=True)
CA_pst.drop_duplicates(inplace=True)
CA_pst.rename(columns = {'Neighborhood_joined': 'Neighborhood'}, inplace=True)
CA_pst.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
CA_pst.shape

(103, 3)

# Problem 2

## Get the latitude and the longitude coordinates of each neighborhood.
   ### Use the Geocoder package to get geographical coordinates of each postal code from this csv link -http://cocl.us/Geospatial_data

In [27]:
! pip install geocoder
import geocoder
#Iterate longitude and latitude values for all postal codes
def get_geocode(post_code):
    lat_lng_coords = None
    while (lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(post_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [31]:
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
geo_data.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
geo_merge = pd.merge(geo_data, CA_pst, on='PostalCode')
geo_merge.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [35]:
PstCode_data = geo_merge[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
PstCode_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
